⚠️ This project is mandatory for certification bloc #1.

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 


## Company's description 📇

<a href='https://www.kayak.com' target='_blank'>Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href='https://www.bookingholdings.com/' target='_blank'>Booking Holdings</a> which now holds: 

* <a href='https://booking.com/' target='_blank'>Booking.com</a>
* <a href='https://kayak.com/' target='_blank'>Kayak</a>
* <a href='https://www.priceline.com/' target='_blank'>Priceline</a>
* <a href='https://www.agoda.com/' target='_blank'>Agoda</a>
* <a href='https://Rentalcars.com/' target='_blank'>RentalCars</a>
* <a href='https://www.opentable.com/' target='_blank'>OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href='https://one-week-in.com/35-cities-to-visit-in-france/' target='_blank'>One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
['Mont Saint Michel',
'St Malo',
'Bayeux',
'Le Havre',
'Rouen',
'Paris',
'Amiens',
'Lille',
'Strasbourg',
'Chateau du Haut Koenigsbourg',
'Colmar',
'Eguisheim',
'Besancon',
'Dijon',
'Annecy',
'Grenoble',
'Lyon',
'Gorges du Verdon',
'Bormes les Mimosas',
'Cassis',
'Marseille',
'Aix en Provence',
'Avignon',
'Uzes',
'Nimes',
'Aigues Mortes',
'Saintes Maries de la mer',
'Collioure',
'Carcassonne',
'Ariege',
'Toulouse',
'Montauban',
'Biarritz',
'Bayonne',
'La Rochelle']
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

# Let's start

## Get weather data with an API

In [ ]:
# Import list of cities

cities = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle",
]


In [ ]:
import pandas as pd

# Creating a dataframe from the list of cities
df_cities = pd.DataFrame(data=cities, columns=["city"])
df_cities = df_cities.reset_index()
df_cities = df_cities.rename(columns={"index": "id"})
df_cities


### Nominatim API : get the gps coordinates of all the cities => https://nominatim.org/release-docs/latest/

In [ ]:
# Try a city with blank space in name to see if formatting needed:
import requests

params = {"city": "La Rochelle", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


In [ ]:
# I don't know 'Chateau du Haut Koenigsbourg', check if Nominatim knows it
import requests

params = {"city": "Chateau du Haut Koenigsbourg", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


In [ ]:
# 'Gorges du Verdon' isn't a city but a canyon, check if Nominatim works with or not
import requests

params = {"city": "Gorges du Verdon", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r

# Doesnt' works... Google search : 'Castellane' will be use for Gorges du Verdon


Ariege is a department and not a city, its prefecture is 'Foix'. We'll use this city

#### Request API

In [ ]:
# Creating a copy of df_cities to store the coordinates from Nominatim API

df_gps = df_cities.copy()
lat_list = []
lon_list = []

for i in cities:
    print(f"Request for city: {i}")
    params = {"city": i, "country": "France", "format": "json"}
    # No 'Gorges du Verdon' city, replacing by 'Castellane'
    if i == "Gorges du Verdon":
        i = "Castellane"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    # 'Ariege' not a city, using the prefecture instead -> 'Foix'
    elif i == "Ariege":
        i = "Foix"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    else:
        r = requests.get(f"https://nominatim.openstreetmap.org/search?", params).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])

# Adding the coordinates to the dataframe
df_gps["lat"] = lat_list
df_gps["lon"] = lon_list


In [ ]:
print(df_gps.head())

print()

print(df_gps.info())


### OpenWeather : get weather of the week => https://openweathermap.org

In [ ]:
# try at St Malo
parameters = {
    "lat": 48.649518,
    "lon": -2.0260409,
    "exclude": "current,minutely,hourly",
    "units": "metric",
    "appid": "aa423e6694bf72625fe1fe31544949dc",
    "lang": "fr",
}

r = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall", params=parameters
).json()

r


In [ ]:
# Weather in 3 days at St Malo
r["daily"][3:]


In [ ]:
day3 = r["daily"][3]  # Weather in 3 days

# desciption of the main weather in 3 days at St Malo
day3["weather"][0]["main"]


In [ ]:
import numpy as np

df_weather = df_gps.copy(deep=True)
my_api_key = "aa423e6694bf72625fe1fe31544949dc"

temperatures_list = []
rain_list = []
weather_list = []

days = list(range(1, 8))

for i in df_weather.itertuples():
    lat = i.lat
    lon = i.lon

    parameters = {
        "lat": {lat},
        "lon": {lon},
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": "aa423e6694bf72625fe1fe31544949dc",
        "lang": "fr",
    }

    r = requests.get(
        f"https://api.openweathermap.org/data/2.5/onecall?", parameters
    ).json()
    forecast_7days = r["daily"][
        1:
    ]  # Getting the weather data for the next 7 days, first item is the current weather, which we don't want here
    temperatures = [int(d["feels_like"]["day"]) for d in forecast_7days]
    rain = [int(d["pop"] * 100) for d in forecast_7days]
    weather = [str(d["weather"][0]["main"]) for d in forecast_7days]
    temperatures_list.append(temperatures)
    rain_list.append(rain)
    weather_list.append(weather)

df_weather["jour_+x"] = [days for _ in range(len(df_weather))]
df_weather["temperature_ressentie"] = temperatures_list
df_weather["probabilite_de_pluie"] = rain_list
df_weather["meteo_principale"] = weather_list
df_weather["score"] = df_weather.apply(
    lambda x: ((np.mean(x["temperature_ressentie"])) * 2)
    + np.mean(x["probabilite_de_pluie"]),
    axis=1,
).astype(int)


print(df_weather.head())

print()

print(df_weather.info())


In [ ]:
df_weather[["lat", "lon"]] = df_weather[["lat", "lon"]].astype(float)


In [ ]:
df = df_weather.apply(pd.Series.explode)  # To obtain a line per day and per city

print(df.head())

print()

print(df.info())


In [ ]:
df[["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]] = df[
    ["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]
].astype(int)


In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    hover_name="city",
    zoom=4,
    hover_data=["meteo_principale", "probabilite_de_pluie", "temperature_ressentie"],
    color="temperature_ressentie",
    color_continuous_scale="thermal",
    mapbox_style="carto-positron",
)
fig.show()


## Scrape Booking.com 

### Get hotels, their URLs, their notes and their description

In [ ]:
import scrapy


class BookingSpider(scrapy.Spider):
    # Name of my spider
    name = "booking_spider"
    cities = df_weather.city
    # Url to start my spider from
    start_urls = [
        "https://www.booking.com/index.fr.html",
    ]

    # Callback function that will be called when starting my spider
    def parse(self, response):
        for x in cities:
            yield scrapy.FormRequest.from_response(
                response, formdata={"ss": x}, callback=self.after_search
            )

    def after_search(self, response):
        cities = response.url.split("ss=")[-1].split("&")[0]
        booking = response.css(".d4924c9e74")

        for data in booking:

            yield {
                "ville": cities,
                "hotels": data.css("a div.fcab3ed991.a23c043802::text").getall(),
                "liens": data.css("h3.a4225678b2 a::attr(href)").getall(),
                #'note': data.css('div.b5cd09854e.d10a6220b4::text').getall(),
                #'description' : data.css ('div.d8eab2cf7f::text').getall()
            }

        try:
            next_page = response.css("a.paging-next").attrib["href"]
        except KeyError:
            logging.info("No next page. Terminating crawling process.")
        else:
            yield response.follow(next_page, callback=self.after_search)

from scrapy.crawler import CrawlerProcess
import logging
import os

# Name of the file where the results will be saved
filename = "hotels.json"

# If file already exists, delete before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir():
    os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(
    settings={
        "USER_AGENT": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1",
        "LOG_LEVEL": logging.INFO,
        "FEEDS": {
            filename: {"format": "json"},
        },
        "AUTOTHROTTLE_ENABLED": True,
    }
)

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()



### Get hotels' coordinates

In [1]:
# Create a DataFrame from the json file
import pandas as pd

df_scraped = pd.read_json("hotels.json")
df_scraped.head()


,ville,hotels,liens
0,Mont+Saint+Michel,"[Hôtel Vert, Les Terrasses Poulard, Mercure Mo...",[https://www.booking.com/hotel/fr/vert.fr.html...
1,St+Malo,"[Ambassadeurs Logis Hotel, Residence Reine Mar...",[https://www.booking.com/hotel/fr/ambassadeurs...
2,Bayeux,"[ibis budget Bayeux, Domaine de Bayeux, Châtea...",[https://www.booking.com/hotel/fr/etap-bayeux....
3,Le+Havre,"[Hôtel Le Marignan - Le Havre Centre Gare, Hol...",[https://www.booking.com/hotel/fr/la-baraka.fr...
4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Mercure Rou...",[https://www.booking.com/hotel/fr/radisson-blu...


In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
import logging
import os


class CoordsSpider(scrapy.Spider):
    # Name of my spider
    name = 'coords_spider'
    cities = df_scraped.ville

    # Url to start my spider from
    start_urls = []
    for i in range(0,35):
        start_urls += df_scraped['liens'][i]

    # Callback function that will be called when starting my spider
    def parse(self, response):

        yield {
            #'ville': cities,
            'lat' : (response.css('p.address.address_clean a::attr(data-atlas-latlng)').get().split(",")[0]),
            'lon' : (response.css('p.address.address_clean a::attr(data-atlas-latlng)').get().split(",")[1])
        }



# Name of the file where the results will be saved
filename = "hotels_coords.json"

# If file already exists, delete before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir():
    os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(
    settings={
        "USER_AGENT": "Mozilla/5.0 (Windows NT 5.1; rv:11.0) Gecko Firefox/11.0",
        "LOG_LEVEL": logging.INFO,
        "FEEDS": {
            filename: {"format": "json"},
        },
        "AUTOTHROTTLE_ENABLED": True,
    }
)

# Start the crawling using the spider you defined above
process.crawl(CoordsSpider)
process.start()
        

2022-05-24 20:19:27 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-24 20:19:27 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  5 2022, 01:53:17) - [Clang 12.0.0 ], pyOpenSSL 21.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 3.4.8, Platform macOS-10.16-x86_64-i386-64bit
2022-05-24 20:19:27 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 5.1; rv:11.0) Gecko Firefox/11.0'}
2022-05-24 20:19:27 [scrapy.extensions.telnet] INFO: Telnet Password: 8c36be2119269e21
2022-05-24 20:19:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.throttle.AutoThrottle']
20

In [3]:
coords = pd.read_json("hotels_coords.json")
coords

,lat,lon
0,48.614700,-1.509617
1,48.635349,-1.510379
2,48.614247,-1.510545
3,48.617587,-1.510396
4,48.616882,-1.510918
...,...,...
870,45.746023,4.823894
871,45.737051,4.837399
872,45.757690,4.858226
873,45.749297,4.829564


## Create your data lake using S3

## ETL